In [ ]:
#Iterative final size calculations for leaky vaccine scenario.

    #Parameters:
    #R0 reproduction number.
    #s0 susceptibility at start season.
    #Vc vaccine coverage.
    #Ve vaccine efficacy.
    
    #z_u final size unvaccinated (f_u final size function unvaccinated).
    #z_v final size vaccinated (f_v final size function vaccinated).

import numpy as np

def finalSizeFunctionSusceptibles(R0,s0,zS,zP):
    return s0*( 1 - np.exp(-R0*( zS + zP )) )

def finalSizeFunctionVaccinates(R0,p0,Vr,zS,zP):
    return p0*( 1 - np.exp(-R0*(1-Vr)*( zS + zP )) )

def findFinalSizeLeakyVaccine(R0,s0,p0,Vr):
    
    zSold = finalSizeFunctionSusceptibles(R0,s0,1,1)
    zPold = finalSizeFunctionVaccinates(R0,p0,Vr,1,1)
    
    stepChange = 1
    while(stepChange > 0.00000001):
        zSnew = finalSizeFunctionSusceptibles(R0,s0,zSold,zPold)
        zPnew = finalSizeFunctionVaccinates(R0,p0,Vr,zSold,zPold)
        
        stepChange = abs(zSold-zSnew) + abs(zPold-zPnew)
        zSold = zSnew
        zPold = zPnew
    
    return zSnew, zPnew

In [1]:
def giveNextSusceptibility(R0,gamma,Vc,Ve,Vr,sIn,pIn):
    #First vaccination, then epidemic, at last waning.
    s1 = (1.0-Vc*Ve)*sIn
    p1 = pIn + Vc*Ve*sIn
    
    zS, zP = findFinalSizeLeakyVaccine(R0,s1,p1,Vr)
    
    s2 = s1 - zS
    p2 = p1 - zP
    
    sOut = 1.0 - gamma*( 1.0 - s2 )
    pOut = gamma*p2
    
    #f = fraction that is vaccinated this year.
    #if p1 == 0.0:
    #    vaccineEfficacy = Ve
    #else:
    #    if Vc*Ve*sIn == 0.0: f = 0.0
    #    else: f = (Vc*Ve*sIn)/p1

    #    vaccineEfficacy = 1.0 - zP/zS
    
    #Return final size in susceptible/vaccinated population.
    return (zS/s1,zP/p1),(sOut,pOut) #,vaccineEfficacy

In [ ]:
#Timeseries functions.
import random as rnd
import numpy as np

def calculateTimeseries(parameters,years,mode):
    
    #Add burn-in period.
    years = range(0,len(years)+50)
    
    #Initialise output vectors (s: susceptibles, p: partly immunes).
    timeseriesSusc = dict([('s',[1.0]),('p',[0.0])])      #Fraction susceptibles at start of season.
    timeseriesFinal = dict([('s',[0.0]),('p',[0.0])])     #Fraction infected during seasonal epidemic.
    
    for year in years:
        
        if mode == 'Deterministic':
            gamma = parameters['gamma']
        elif mode == 'Stochastic':
            gamma = rnd.betavariate(5,2)
        else:
            print 'Wrong mode. Set mode to Deterministic or Stochastic.'
        
        s, p = timeseriesSusc['s'][-1], timeseriesSusc['p'][-1]
        
        finalSize, susceptibility = giveNextSusceptibility(parameters['R0'],gamma,
                                                           parameters['Vc'],
                                                           parameters['Ve'],parameters['Vr'],s,p)
        
        timeseriesSusc['s'] += [ susceptibility[0] ]
        timeseriesSusc['p'] += [ susceptibility[1] ]
        
        timeseriesFinal['s'] += [ finalSize[0] ]
        timeseriesFinal['p'] += [ finalSize[1] ]
   
    #Remove burn-in period.
    timeseriesSusc['s'] = np.array(timeseriesSusc['s'][51:])
    timeseriesSusc['p'] = np.array(timeseriesSusc['p'][51:])
    
    timeseriesFinal['s'] = np.array(timeseriesFinal['s'][51:])
    timeseriesFinal['p'] = np.array(timeseriesFinal['p'][51:])
        
    return timeseriesSusc, timeseriesFinal

In [2]:
#Functions below are the same as in Functions of the main folder (all-or-nothing vaccine).

In [ ]:
#SIR-model for one season.
import numpy as np
from scipy.integrate import odeint

def SPIR(R0,Vr,s0,p0):
    nu = 0.5
    beta = nu*R0
    N = 1000000.0
    stepsSeason = 1000
        
    if N*s0 < 100:
        print 'Warning: N to small'
    
    def derivative(y,t):
        #S=y[0], P=y[1] I=y[2], R=y[3]
        
        dSdt = -(beta/N)*y[0]*y[2]
        dPdt = -(beta/N)*(1-Vr)*y[1]*y[2]
        
        dIdt = (beta/N)*(y[0]+(1-Vr)*y[1])*y[2] - nu*y[2]
        dRdt = nu*y[2]
        
        return [dSdt, dPdt, dIdt, dRdt]

    endTime = 1000
    time = np.linspace(0.0,endTime,stepsSeason)
    yinit = np.array([(s0-0.00001)*N,(p0-0.00001)*N,0.00002*N,(1-s0-p0)*N])

    y = odeint(derivative,yinit,time)
    
    proportions = np.array([[y[i,0]/N, y[i,1]/N, y[i,2]/N, y[i,3]/N] for i in range(len(y))])
    return time, proportions

In [ ]:
#Peak incidence function.

def peakIncidence(R0,Vr,s0,p0):
    timeSeason, y = SPIR(R0,Vr,s0,p0)
    return max(y[:,2])

In [ ]:
#Plot function. Removes box around plot.
def removeFrame(ax):
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    for tic in ax.xaxis.get_major_ticks():
        tic.tick2On = False
    for tic in ax.yaxis.get_major_ticks():
        tic.tick2On = False

In [ ]:
#Save data to file for later import back into python.
import pickle

def writeData(filename,data):
    outfile = open(filename + '.dat', 'w' )
    pickle.dump(data, outfile)
    outfile.close()

#Load data again.
def readData(filename):
    infile = open(filename + '.dat', 'r')
    return pickle.load(infile)